In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='/Users/hoon/Library/Fonts/NanumSquareRegular.ttf').get_name()
plt.rc("font", family=font_name)
import matplotlib as mpl
mpl.rcParams["axes.unicode_minus"] = False

import requests as req
import time
import re


from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV, KFold # 데이터 나누기, 점검 및 훈련

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_score, recall_score # 평가지표
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler # Feature Scaling


from sklearn.neighbors import KNeighborsRegressor

from sklearn.datasets import load_iris, load_boston, load_breast_cancer, make_moons, load_digits, load_diabetes, load_wine, make_blobs # 데이터
import mglearn # 그래프

from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC

# conda install -c conda -py
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance, plot_tree

from sklearn.pipeline import make_pipeline
import multiprocessing
from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPRegressor

# pip install lightgbm
# conda install -c conda-forge lightgbm
from lightgbm import LGBMRegressor
from lightgbm import plot_importance, plot_tree, plot_metric

import graphviz

import statsmodels.api as sm

# SVC는 분류, SVR은 회귀
# 분류알고리즘에선 끝판왕으로 불림
from sklearn.svm import SVR, SVC

## 1. 데이터 불러오기

In [33]:
lunch = pd.read_csv("data/lunch_df_encoding.csv")
dinner = pd.read_csv("data/dinner_df_encoding.csv")

In [34]:
lunch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   datetime               1205 non-null   object 
 1   year                   1205 non-null   int64  
 2   month                  1205 non-null   int64  
 3   date                   1205 non-null   int64  
 4   worker_number          1205 non-null   int64  
 5   real_number            1205 non-null   int64  
 6   vacation_number        1205 non-null   int64  
 7   biztrip_number         1205 non-null   int64  
 8   overtime_number        1205 non-null   int64  
 9   telecom_number         1205 non-null   int64  
 10  temperature            1204 non-null   float64
 11  rain                   1205 non-null   float64
 12  wind                   1204 non-null   float64
 13  humidity               1205 non-null   float64
 14  discomfort_index       1204 non-null   float64
 15  perc

In [35]:
dinner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   datetime               1205 non-null   object 
 1   year                   1205 non-null   int64  
 2   month                  1205 non-null   int64  
 3   date                   1205 non-null   int64  
 4   worker_number          1205 non-null   int64  
 5   real_number            1205 non-null   int64  
 6   vacation_number        1205 non-null   int64  
 7   biztrip_number         1205 non-null   int64  
 8   overtime_number        1205 non-null   int64  
 9   telecom_number         1205 non-null   int64  
 10  temperature            1204 non-null   float64
 11  rain                   1205 non-null   float64
 12  wind                   1204 non-null   float64
 13  humidity               1205 non-null   float64
 14  discomfort_index       1204 non-null   float64
 15  perc

In [36]:
lunch = lunch.dropna()
dinner = dinner.dropna()

In [37]:
def split_lunch(data):
    data_train = data.drop(["lunch_number", "datetime"], axis = "columns")
    data_target = data.loc[:, 'lunch_number']
    X_train, X_test, y_train, y_test = train_test_split(data_train, data_target)

    return X_train, X_test, y_train, y_test

def split_dinner(data):
    data_train = data.drop(["dinner_number", "datetime"], axis = "columns")
    data_target = data.loc[:, 'dinner_number']
    X_train, X_test, y_train, y_test = train_test_split(data_train, data_target)

    return X_train, X_test, y_train, y_test

In [38]:
lunch_X_train, lunch_X_test, lunch_y_train, lunch_y_test = split_lunch(lunch)
dinner_X_train, dinner_X_test, dinner_y_train, dinner_y_test = split_dinner(dinner)

In [39]:
lunch_X_train.head()

,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,temperature,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_lunch_N,new_lunch_Y
354,2017,7,10,2839,2526,98,215,406,0,26.9,...,0,0,1,0,0,0,1,0,1,0
943,2019,12,12,3111,2664,190,257,586,0,2.9,...,1,0,0,1,0,0,1,0,1,0
717,2019,1,11,2985,2557,177,251,1,0,2.4,...,1,1,0,0,0,0,1,0,1,0
1045,2020,5,14,2962,2398,71,281,488,212,17.2,...,0,0,0,1,0,0,1,0,1,0
137,2016,8,23,2644,2347,86,211,386,0,26.8,...,0,0,0,0,1,0,1,0,1,0


In [47]:
lunch_X_test.head()

,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,temperature,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_lunch_N,new_lunch_Y
650,2018,9,21,2763,2262,260,241,51,0,19.4,...,0,1,0,0,0,0,0,1,1,0
906,2019,10,22,3121,2644,99,378,381,0,15.9,...,0,0,0,0,1,0,1,0,1,0
41,2016,4,4,2643,2360,62,221,367,0,11.2,...,0,0,1,0,0,0,1,0,1,0
101,2016,7,1,2643,2283,145,215,41,0,23.9,...,0,1,0,0,0,0,1,0,1,0
1089,2020,7,31,2997,2115,496,261,0,125,26.2,...,0,1,0,0,0,0,1,0,1,0


In [40]:
dinner_X_train.head()

,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,temperature,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_dinner_N,new_dinner_Y
748,2019,2,28,2806,2281,254,271,116,0,8.5,...,1,0,0,1,0,0,0,1,1,0
464,2017,12,20,2665,2295,108,262,0,0,-1.9,...,1,0,0,0,0,1,1,0,1,0
624,2018,8,16,2996,2287,481,228,228,0,25.6,...,0,0,0,1,0,0,1,0,1,0
889,2019,9,24,3111,2775,71,265,631,0,18.3,...,0,0,0,0,1,0,1,0,1,0
973,2020,1,29,2821,2506,101,214,4,0,6.6,...,1,0,0,0,0,1,1,0,1,0


## 2. LinearRegression

### 점심

In [41]:
lr = LinearRegression()
lr.fit(lunch_X_train, lunch_y_train)

print("기울기 : {0}, 절편 : {1}".format(lr.coef_, lr.intercept_))
print("훈련 설명력 : {}".format(lr.score(lunch_X_train, lunch_y_train)))
print("테스트 설명력 : {}".format(lr.score(lunch_X_test, lunch_y_test)))

기울기 : [-2.07369296e+01 -7.04310138e+00 -5.64956788e-01 -3.42087788e+12
  3.42087788e+12  3.42087788e+12  3.42087788e+12  2.74266250e-01
  3.42087788e+12  1.77456411e+01  1.02127724e+00 -9.15139213e+00
  4.08442034e-01 -3.51843400e-01 -1.67674209e+01 -6.95245577e+08
 -3.04185961e+09 -3.04185966e+09 -3.04185963e+09 -3.04185964e+09
  3.34466474e+09  3.34466495e+09  3.34466473e+09  3.34466477e+09
  3.34466488e+09 -9.08282071e+08 -9.08282155e+08 -1.50357742e+09
 -1.50357736e+09], 절편 : 2109097507.5289576
훈련 설명력 : 0.7926581125575963
테스트 설명력 : 0.7724916159106434


In [51]:
lr = LinearRegression()
lr.fit(dinner_X_train, dinner_y_train)

print("기울기 : {0}, 절편 : {1}".format(lr.coef_, lr.intercept_))
print("훈련 설명력 : {}".format(lr.score(dinner_X_train, dinner_y_train)))
print("테스트 설명력 : {}".format(lr.score(dinner_X_test, dinner_y_test)))

기울기 : [-2.68468279e+01 -3.11575501e+00 -1.82139069e+00 -2.02609973e+13
  2.02609973e+13  2.02609973e+13  2.02609973e+13  3.24091074e-01
  2.02609973e+13  1.21832531e+01  3.20323469e-01 -1.08960645e+00
 -3.45940125e-01 -3.51009985e-01 -9.62973853e+00 -1.09505056e+09
 -2.56039559e+10 -2.56039559e+10 -2.56039559e+10 -2.56039559e+10
 -3.13090055e+10 -3.13090055e+10 -3.13090055e+10 -3.13090055e+10
 -3.13090055e+10  2.91150856e+09  2.91150853e+09  1.52476783e+10
  1.52476783e+10], 절편 : 38753829348.495735
훈련 설명력 : 0.5296821747972162
테스트 설명력 : 0.5083136302819795


## 3. SVR

### 점심

In [58]:
pipe = Pipeline([('standardscaler', StandardScaler()), ("model", SVR(kernel="linear"))])
params = [{"model__gamma":['scale', 'auto'], 'model__C':[1.0, 0.1, 0.01]}]

gs = GridSearchCV(estimator=pipe, param_grid=params, n_jobs=multiprocessing.cpu_count(), verbose=True, cv=5)

gs.fit(lunch_X_train, lunch_y_train)

print(gs.best_params_)
print(gs.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'model__C': 1.0, 'model__gamma': 'scale'}
0.7640825757935178


In [53]:
model = SVR(kernel='linear', C=1.0, gamma='scale')
model.fit(lunch_X_train, lunch_y_train)

print("훈련 결과 : {}".format(model.score(lunch_X_train, lunch_y_train)))
print("점검 결과 : {}".format(model.score(lunch_X_test, lunch_y_test)))

훈련 결과 : 0.7597457770278903
점검 결과 : 0.7370542386595125


### 저녁

In [54]:
pipe = Pipeline([('standardscaler', StandardScaler()), ("model", SVR(kernel="linear"))])
params = [{"model__gamma":['scale', 'auto'], 'model__C':[1.0, 0.1, 0.01]}]

gs = GridSearchCV(estimator=pipe, param_grid=params, n_jobs=multiprocessing.cpu_count(), verbose=True, cv=5)

gs.fit(dinner_X_train, dinner_y_train)

print(gs.best_params_)
print(gs.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'model__C': 1.0, 'model__gamma': 'scale'}
0.4435641681737881


In [55]:
model = SVR(kernel='linear', C=1.0, gamma='scale')
model.fit(dinner_X_train, dinner_y_train)

print("훈련 결과 : {}".format(model.score(dinner_X_train, dinner_y_train)))
print("점검 결과 : {}".format(model.score(dinner_X_test, dinner_y_test)))

훈련 결과 : 0.4707609813253647
점검 결과 : 0.4659953503973886


## 4. XGBoost

### 점심

In [64]:
param = {
    'max_depth':[2,3,4],
    'n_estimators':range(300,600,100), #  'n_estimators':range(600,700,50) 여기에 cv 10 (이거와 별반차이가 없다.)
    'colsample_bytree':[0.5,0.7,1],
    'colsample_bylevel':[0.5,0.7,1]
}

model = XGBRegressor()
gs = GridSearchCV(estimator=model, param_grid=param, cv=10, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)
print(gs.best_params_)
print(gs.best_score_)

{'colsample_bylevel': 0.7, 'colsample_bytree': 0.5, 'max_depth': 2, 'n_estimators': 300}
-8543.811321200254


In [69]:
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
               colsample_bytree=0.7, gamma=0, learning_rate=0.1, max_delta_step=0,
               max_depth=2, min_child_weight=1, missing=None, n_estimators=300,
               n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
               silent=True, subsample=1)

model.fit(lunch_X_train, lunch_y_train)
print("훈련 결과 : {}".format(model.score(lunch_X_train, lunch_y_train)))
print("점검 결과 : {}".format(model.score(lunch_X_test, lunch_y_test)))

[00:45:03] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[00:45:03] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBoostError: [00:45:03] ../src/c_api/c_api_utils.h:161: Invalid missing value: null
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000016030f424 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x0000000160308ecc xgboost::GetMissing(xgboost::Json const&) + 364
  [bt] (2) 3   libxgboost.dylib                    0x00000001603165b4 void InplacePredictImpl<xgboost::data::ArrayAdapter>(std::__1::shared_ptr<xgboost::data::ArrayAdapter>, std::__1::shared_ptr<xgboost::DMatrix>, char const*, xgboost::Learner*, unsigned long, unsigned long, unsigned long long const**, unsigned long long*, float const**) + 496
  [bt] (3) 4   libxgboost.dylib                    0x0000000160316074 XGBoosterPredictFromDense + 320
  [bt] (4) 5   libffi.dylib                        0x00000001ad974050 ffi_call_SYSV + 80
  [bt] (5) 6   libffi.dylib                        0x00000001ad97c9e4 ffi_call_int + 948
  [bt] (6) 7   _ctypes.cpython-39-darwin

### 저녁

In [ ]:
param = {
    'max_depth':[2,3,4],
    'n_estimators':range(300,600,100), #  'n_estimators':range(600,700,50) 여기에 cv 10 (이거와 별반차이가 없다.)
    'colsample_bytree':[0.5,0.7,1],
    'colsample_bylevel':[0.5,0.7,1]
}

model = XGBRegressor()
gs = GridSearchCV(estimator=model, param_grid=param, cv=10, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

gs.fit(dinner_X_train, dinner_y_train)
print(gs.best_params_)
print(gs.best_score_)

In [ ]:
pred = gs.predict(dinner_X_test)
print("테스트 정확도 : {}".format(accuracy_score(dinner_y_test, pred)))